In [1]:
import pandas as pd

In [2]:
df = pd.read_csv(r'sota_data.csv')
df.head()

,serial_number,model,capacity_bytes,hard_live_cost,smart_1_normalized,smart_2_normalized,smart_3_normalized,smart_4_normalized,smart_5_normalized,smart_6_normalized,...,smart_246_raw,smart_247_raw,smart_248_raw,smart_249_raw,smart_250_raw,smart_251_raw,smart_252_raw,smart_253_raw,smart_254_raw,smart_255_raw
0,PL1331LAGDJ5GH,HGST HDS5C4040ALE630,4000787030016,2088,100.0,134.0,139.0,100.0,100.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,PL2331LAGLW5UJ,HGST HMS5C4040BLE640,4000787030016,2088,100.0,133.0,127.0,100.0,100.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,PL2331LAGM410J,HGST HMS5C4040BLE640,4000787030016,2088,100.0,134.0,124.0,100.0,100.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,PL2331LAGM5NTJ,HGST HMS5C4040BLE640,4000787030016,2088,100.0,135.0,121.0,100.0,100.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,PL2331LAGM5R4J,HGST HMS5C4040BLE640,4000787030016,2088,100.0,135.0,123.0,100.0,100.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df.columns

Index(['serial_number', 'model', 'capacity_bytes', 'hard_live_cost',
       'smart_1_normalized', 'smart_2_normalized', 'smart_3_normalized',
       'smart_4_normalized', 'smart_5_normalized', 'smart_6_normalized',
       ...
       'smart_246_raw', 'smart_247_raw', 'smart_248_raw', 'smart_249_raw',
       'smart_250_raw', 'smart_251_raw', 'smart_252_raw', 'smart_253_raw',
       'smart_254_raw', 'smart_255_raw'],
      dtype='object', length=514)

In [4]:
data = df

In [5]:
from sklearn.model_selection import train_test_split

data = df
data = data.drop(columns=['serial_number'])
threshold = 0.5  # Порог: 50%
data = data.loc[:, data.isnull().mean() < threshold]

X = data.drop(columns=['hard_live_cost'])
y = data['hard_live_cost']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
data.isna().sum()

model                        0
capacity_bytes               0
hard_live_cost               0
smart_1_normalized         554
smart_2_normalized      116781
smart_3_normalized        3075
smart_4_normalized        3075
smart_5_normalized        2318
smart_7_normalized        3075
smart_8_normalized      116781
smart_9_normalized         441
smart_10_normalized       3075
smart_12_normalized        441
smart_192_normalized      1198
smart_193_normalized      3470
smart_194_normalized       441
smart_196_normalized    115974
smart_197_normalized      8397
smart_198_normalized      2431
smart_199_normalized      2318
smart_240_normalized     79115
smart_241_normalized    116036
smart_242_normalized    116038
smart_1_raw                554
smart_2_raw             116781
smart_3_raw               3075
smart_4_raw               3075
smart_5_raw               2318
smart_7_raw               3075
smart_8_raw             116781
smart_9_raw                441
smart_10_raw              3075
smart_12

In [7]:
from autogluon.tabular import TabularPredictor
# Разделение данных на обучающие и тестовые
data = data.sparse.to_dense()
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Повторное обучение на обучающих данных
predictor = TabularPredictor(
    label = 'hard_live_cost',
    problem_type = 'regression',
    eval_metric = 'symmetric_mean_absolute_percentage_error',
    ).fit(train_data, time_limit = 600, presets = 'interpretable')

# Оценка модели на тестовой выборке
performance = predictor.evaluate(test_data)
print(performance)

c:\Users\Goldian\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No path specified. Models will be saved in: "AutogluonModels\ag-20240907_164728"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.11.4
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19045
CPU Count:          12
Memory Avail:       20.37 GB / 31.95 GB (63.7%)
Disk Space Avail:   14.94 GB / 232.24 GB (6.4%)
Presets specified: ['interpretable']
Beginning AutoGluon training ... Time limit = 600s
AutoGluon will save models to "AutogluonModels\ag-20240907_164728"
Train Data Rows:    194135
Train Data Columns: 42
Label Column:       hard_live_cost
Problem Type:       regression
Preprocessing data ...
Usin

In [7]:
from catboost import CatBoostRegressor

categorical_features = ['model']
# Инициализация модели
model = CatBoostRegressor(iterations=100, 
                          learning_rate=0.1, 
                          depth=12, 
                          loss_function='MAE', 
                          random_seed=42)

# Обучение модели
model.fit(X_train, y_train, 
          cat_features=categorical_features, 
          eval_set=(X_test, y_test),
          verbose=100,
          early_stopping_rounds=50)


0:	learn: 9.6917450	test: 10.6045881	best: 10.6045881 (0)	total: 521ms	remaining: 51.6s


KeyboardInterrupt: 